<a href="https://colab.research.google.com/github/Abhishek3253/VN2/blob/main/assignment06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz


In [5]:
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [6]:

!pip install -q findspark

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [8]:
import findspark
findspark.init()

In [9]:
findspark.find()

'/content/spark-3.2.1-bin-hadoop3.2'

In [10]:
from pyspark.sql import SparkSession

In [11]:

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [14]:
df = spark.read.csv(r"/content/assignment.csv", header=True, inferSchema=True)

In [ ]:
df.show()

+------+----------------+----------+------+-----------+
|userid|         emailid|      date|amount|   discount|
+------+----------------+----------+------+-----------+
|     1| test1@gmail.com|01-01-2019|   100|        1.0|
|     2| test2@gmail.com|02-01-2019|   100|        1.0|
|     3| test3@gmail.com|03-01-2019|   101|        2.0|
|     4| test4@gmail.com|04-01-2019|   102|       10.0|
|     5| test5@gmail.com|05-01-2019| 102.5|        2.0|
|     6| test6@gmail.com|06-01-2019| 103.2|        1.0|
|     7| test7@gmail.com|07-01-2019| 103.9|        3.0|
|     8| test8@gmail.com|08-01-2019| 104.6|        5.0|
|     9| test9@gmail.com|09-01-2019| 105.3|4.571428571|
|    10|test10@gmail.com|10-01-2019|   106|4.892857143|
|    11|test11@gmail.com|11-01-2019| 106.7|5.214285714|
|    12|test12@gmail.com|12-01-2019| 107.4|5.535714286|
|    13|test13@gmail.com|13-01-2019| 108.1|5.857142857|
|    14|test14@gmail.com|14-01-2019| 108.8|6.178571429|
|    15|test15@gmail.com|15-01-2019| 109.5|     

In [15]:
df.count()

902

In [16]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [19]:
df1 = df.dropna()

In [20]:
#count after drop null 

df1.count()

901

In [21]:
#count before droping null
df.count()

902

In [22]:
#to print null value 
df2 =df.filter(df.date == 'null')

In [23]:
df2.show()

+------+----------------+----+------+--------+
|userid|         emailid|date|amount|discount|
+------+----------------+----+------+--------+
|    78|test78@gmail.com|null| 258.6|   26.75|
+------+----------------+----+------+--------+



In [39]:
#UDF function to check date format
from datetime import datetime
def func_date(str1):
  for fmt in ('%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y','%d-%m-%Y','%dd-%mm-%yyyy'):
    try:
      date_object =  datetime.strptime(str1, fmt)
      converted_date = date_object.strftime('%d-%m-%Y')
      return converted_date
    except ValueError:
      pass
  raise ValueError('no valid date format found')

In [40]:
date_udf= udf(lambda x: func_date(x))

In [41]:
date_change = df.select(col('date'),date_udf(col('date').alias("Updated_date")))


In [42]:
date_change.show()

+----------+------------------------------+
|      date|<lambda>(date AS Updated_date)|
+----------+------------------------------+
|01-01-2019|                    01-01-2019|
|02-01-2019|                    02-01-2019|
|03-01-2019|                    03-01-2019|
|04-01-2019|                    04-01-2019|
|05-01-2019|                    05-01-2019|
|06-01-2019|                    06-01-2019|
|07-01-2019|                    07-01-2019|
|08-01-2019|                    08-01-2019|
|09-01-2019|                    09-01-2019|
|10-01-2019|                    10-01-2019|
|11-01-2019|                    11-01-2019|
|12-01-2019|                    12-01-2019|
|13-01-2019|                    13-01-2019|
|14-01-2019|                    14-01-2019|
|15-01-2019|                    15-01-2019|
|16-01-2019|                    16-01-2019|
|17-01-2019|                    17-01-2019|
|18-01-2019|                    18-01-2019|
|19-01-2019|                    19-01-2019|
|20-01-2019|                    

In [71]:
!pip install validate_email

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for validate-email: filename=validate_email-1.3-py3-none-any.whl size=5482 sha256=d795e0e8dd786a2d5a2e7a0edd3b262908034a44890a82eabb56cf6a09144509
  Stored in directory: /root/.cache/pip/wheels/ff/8f/92/c43287715852eaa75e0d8aa1941c481072b4a82c4f4975074e
Successfully built validate-email


In [76]:
from validate_email import validate_email
is_valid = validate_email("abhishek@gmail.com")


In [77]:
def email_validation(str1):
  is_valid = validate_email(str1)
  if is_valid:
    return str1
  else:
    "error"

In [78]:
email_df=udf(lambda x:email_validation(x))

In [79]:
email_correction=df.select(col('emailid'),email_df(col('emailid').alias('update_Email')))

In [80]:
email_correction.show()

+----------------+---------------------------------+
|         emailid|<lambda>(emailid AS update_Email)|
+----------------+---------------------------------+
| test1@gmail.com|                  test1@gmail.com|
| test2@gmail.com|                  test2@gmail.com|
| test3@gmail.com|                  test3@gmail.com|
| test4@gmail.com|                  test4@gmail.com|
| test5@gmail.com|                  test5@gmail.com|
| test6@gmail.com|                  test6@gmail.com|
| test7@gmail.com|                  test7@gmail.com|
| test8@gmail.com|                  test8@gmail.com|
| test9@gmail.com|                  test9@gmail.com|
|test10@gmail.com|                 test10@gmail.com|
|test11@gmail.com|                 test11@gmail.com|
|test12@gmail.com|                 test12@gmail.com|
|test13@gmail.com|                 test13@gmail.com|
|test14@gmail.com|                 test14@gmail.com|
|test15@gmail.com|                 test15@gmail.com|
|test16@gmail.com|                 test16@gmai

In [49]:
df.show()

+------+----------------+----------+------+-----------+
|userid|         emailid|      date|amount|   discount|
+------+----------------+----------+------+-----------+
|     1| test1@gmail.com|01-01-2019|   100|        1.0|
|     2| test2@gmail.com|02-01-2019|   100|        1.0|
|     3| test3@gmail.com|03-01-2019|   101|        2.0|
|     4| test4@gmail.com|04-01-2019|   102|       10.0|
|     5| test5@gmail.com|05-01-2019| 102.5|        2.0|
|     6| test6@gmail.com|06-01-2019| 103.2|        1.0|
|     7| test7@gmail.com|07-01-2019| 103.9|        3.0|
|     8| test8@gmail.com|08-01-2019| 104.6|        5.0|
|     9| test9@gmail.com|09-01-2019| 105.3|4.571428571|
|    10|test10@gmail.com|10-01-2019|   106|4.892857143|
|    11|test11@gmail.com|11-01-2019| 106.7|5.214285714|
|    12|test12@gmail.com|12-01-2019| 107.4|5.535714286|
|    13|test13@gmail.com|13-01-2019| 108.1|5.857142857|
|    14|test14@gmail.com|14-01-2019| 108.8|6.178571429|
|    15|test15@gmail.com|15-01-2019| 109.5|     